## Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

In [92]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import *
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

In [100]:
def out(fname, num):
    with open(fname, 'w') as f:
        f.write(str(num))

def out_many(fname, nums):
    with open(fname, 'w') as f:
        for num in nums:
            f.write(str(num)+' ')

In [45]:
data = pd.read_csv('smsspamcollection/SMSSpamCollection.txt', sep='\t', header=None, names=['label', 'text'])

In [46]:
data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [71]:
data['y'] = data.apply(lambda row: 1 if row.label == 'spam' else 0, axis=1)
data['y'][:5]

0    0
1    0
2    1
3    0
4    0
Name: y, dtype: int64

In [72]:
texts = data['text'].values
y = data['y'].values

In [78]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

5.Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [79]:
clf_logreg = LogisticRegression(random_state=2)
scores_logreg = cross_val_score(estimator=clf_logreg, X=X, y=y, cv=10, scoring='f1')
mean_scores_logreg = np.mean(scores_logreg)
print('Logistic Regression score: %f') % mean_scores_logreg
print('Answer 5: %f') % round(mean_scores_logreg, 1)
out('5.txt', round(mean_scores_logreg, 1))

Logistic Regression score: 0.932640
Answer 5: 0.900000


6.А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [80]:
model_logreg = clf_logreg.fit(X=X, y=y)

In [106]:
tests = ['FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB',\
         'FreeMsg: Txt: claim your reward of 3 hours talk time',\
         'Have you visited the last lecture on physics?',\
         'Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$',\
         'Only 99$'
        ]
test_X = vectorizer.transform(tests)
test_results = model_logreg.predict(test_X)
print test_results
out_many('6.txt', test_results)

[1 1 0 0 0]


7.Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [88]:
def fit_count_vectorizer(ngram_range, texts):
    vect = CountVectorizer(ngram_range=ngram_range)
    train_X = vect.fit_transform(texts)
    return train_X
    
def evaluate(clf, X, y):
    scores = cross_val_score(estimator=clf, X=X, y=y, cv=10, scoring='f1')
    return np.mean(scores)

In [104]:
ngram_ranges = [(2,2), (3,3), (1,3)]
scores = []

for ngram_range in ngram_ranges:
    train_X = fit_count_vectorizer(ngram_range, texts)
    score = evaluate(clf_logreg, train_X, y)
    scores.append(score)
    print('Logistic Regression score for (%i,%i)-grams: %f' % (ngram_range[0], ngram_range[1], score))
    
out_many('7.txt', [round(x, 2) for x in scores])

Logistic Regression score for (2,2)-grams: 0.822422
Logistic Regression score for (3,3)-grams: 0.725016
Logistic Regression score for (1,3)-grams: 0.925138


8.Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор. 

In [105]:
clf_NB = MultinomialNB()
train_X = fit_count_vectorizer((1,1), texts)
score = evaluate(clf_NB, train_X, y)
print('Multinomial Naive Bayes score without n-grams: %f' % (score))

scores_NB = []
for ngram_range in ngram_ranges:
    train_X = fit_count_vectorizer(ngram_range, texts)
    score = evaluate(clf_NB, train_X, y)
    scores_NB.append(score)
    print('Multinomial Naive Bayes score for (%i,%i)-grams: %f' % (ngram_range[0], ngram_range[1], score))
    
out_many('8.txt', [round(x, 2) for x in scores_NB])

Multinomial Naive Bayes score without n-grams: 0.927730
Multinomial Naive Bayes score for (2,2)-grams: 0.645502
Multinomial Naive Bayes score for (3,3)-grams: 0.378719
Multinomial Naive Bayes score for (1,3)-grams: 0.888486


9.Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данны

In [110]:
vect = TfidfVectorizer()
train_X = vect.fit_transform(texts)
score_tfidf = evaluate(clf_logreg, train_X, y)
print('Logistic Regression with CountVectorizer score on unigrams: %f') % mean_scores_logreg
print('Logistic Regression with TfidfVectorizer score on unigrams: %f') % score_tfidf
diff = score_tfidf - mean_scores_logreg
print('Score difference: %f') % (diff)
if abs(diff) <= 0.01:
    out('9.txt', 0)
elif diff < 0:
    out('9.txt', -1)
else:
    out('9.txt', 1)

Logistic Regression with CountVectorizer score on unigrams: 0.932640
Logistic Regression with TfidfVectorizer score on unigrams: 0.852860
Score difference: -0.079780
